In [ ]:
!pip install vaderSentiment
!pip install gensim
!pip install newspaper3k

     |████████████████████████████████| 133kB 2.7MB/s 
     |████████████████████████████████| 215kB 2.7MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 7.4MB 4.6MB/s 
     |████████████████████████████████| 92kB 8.4MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3355 sha256=34d1b770aee5d32e32564270141cc82ca8e3f8ad7f68f294332e5666ae6e9f06
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=33ff33078cafbdc84cbc6aa549c6a982ca34328756c1652983f4a1ae8e5c8c49
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13538 sha256=20cb74500d7ad495d8128bb5f35297d3885286c20656c01239ac72a149fe3fa2
  Stored in directory: /root/.cach

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
from newspaper import Article
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
from urllib.parse import urlparse

In [ ]:
def googleSearch(ingr):
    g_clean = []
    url = 'https://www.google.com/search?client=ubuntu&channel=fs&q={}&ie=utf-8&oe=utf-8'.format(ingr)
    try:
            html = requests.get(url)
            if html.status_code==200:
                soup = BeautifulSoup(html.text, 'lxml')
                a = soup.find_all('a')
                for i in a:
                    k = i.get('href')
                    try:
                        m = re.search("(?P<url>https?://[^\s]+)", k)
                        n = m.group(0)
                        rul = n.split('&')[0]
                        domain = urlparse(rul)
                        if(re.search('google.com', domain.netloc)):
                            continue
                        else:
                            g_clean.append(rul)
                    except:
                        continue
    except Exception as ex:
            print(str(ex))
    finally:
            return g_clean

def analyse_sentiments(s_line):
    analyser = SentimentIntensityAnalyzer()

    vs = analyser.polarity_scores(s_line)

    if vs['compound'] >= 0.1:
        return vs['compound']
    elif vs['compound'] <= -0.1:
        return vs['compound']
    else:
        return vs['compound']

def safe_or_not(s):
    if s >= 2.8:
        return 'healthy'
    elif s <= 1.8:
        return 'potentially harmful'
    else:
        return 'neutral'

def avg(lst, n):
    return sum(lst)/n

def scrapper(url):
    article = Article(url)

    article.download()
    article.parse()
    return article.text

In [ ]:
import time

start = time.time()

In [ ]:
lst_of_ingredients = ['parabens', 'sulfates', 'phthalates', 'fragarance', 'formaldehyde', 'phenoxyethanol', 'methanol', 'polyethylene glycols']
health1 = []

for i in lst_of_ingredients:
    search = 'disadvantages of {} for skin'.format(i)
    l = googleSearch(search)
    l = l[:6]

    score = []
    for url in l:
        try:
            score.append(analyse_sentiments(scrapper(url)))
        except:
            pass
    s = avg(score, len(lst_of_ingredients)/3)
    health1.append(s)

In [ ]:
health2 = []

for i in lst_of_ingredients:
    search = 'advantages of {} for skin'.format(i)
    l = googleSearch(search)
    l = l[:6]

    score = []
    for url in l:
        try:
            score.append(analyse_sentiments(summarize(scrapper(url), ratio=0.7)))
        except:
            pass
    s = avg(score, len(lst_of_ingredients)/3)
    health2.append(s)

In [ ]:
end = time.time()

print(end - start)

95.83723640441895


In [ ]:
for i in range(len(lst_of_ingredients)):
    a = health1[i] + health2[i]
    print(lst_of_ingredients[i], safe_or_not(a), 'Confidence :', a)

In [ ]:
for i in range(len(lst_of_ingredients)):
    print(lst_of_ingredients[i], '-', health1[i], health2[i])

In [ ]:
# Quick Summary
import requests
from bs4 import BeautifulSoup
   
for c in lst_of_ingredients:
    search = "is {} safe for hair".format(c)

    url = f"https://www.google.com/search?&q={search}"
   
    req = requests.get(url)
    print(req)

    sor = BeautifulSoup(req.text, "html.parser")
    s = sor.find("div", class_='BNeawe').text

    print(c, '-',s)